<a href="https://colab.research.google.com/github/rz-pb/Bioinformatics-Codes/blob/main/Partial_Digest_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Partial Digest Probem ( PDP )

In [ ]:
# Importing libraries

import numpy as np
from math import factorial
import random
from collections import Counter

In [ ]:
# instance to check the code when it has solution:
number_of_cuts1 = 5
L1 = np.array([ 2 , 2 , 3 , 3 , 4 , 5 , 6 , 7 , 8 , 10])
L_takehome = np.array([1,1,1,2,2,3,3,3,4,4,5,5,6,6,6,9,9,10,11,12,15])
# instance to check the code when it hasn't any solution: (hint: due to definition of Delta operation, 0 could not be an element in L)
number_of_cuts2 = 5
L2 = np.array([ 0 , 2 , 3 , 3 , 4 , 5 , 6 , 7 , 8 , 10])

In [ ]:
def Induced_Difference(y,X) :
  X = np.array(X)
  result = abs(X-y)
  return result

In [ ]:
# Defining a function to fill the Delta multiset as the collection of mutual differences between each set's elements. 

def Delta(myset) :
  mysortedset = np.sort(myset)

  Deltaset = np.array([])
  
  for i in range(0,len(mysortedset)-1) :
    for j in range(i+1 ,len(mysortedset)) :
      Deltaset = np.append(Deltaset,int(mysortedset[j]-mysortedset[i]))

  return np.sort(Deltaset)

In [ ]:
# This Function simply converts a decimal integer (in base 10) to its binary version ( in base 2) and represents it as a list of 0 s and 1 s , 
# such that the least significant bit would be stored in the the lowest index and so forth. It gets "max_num_pows_of_2" as the maximum number of powers of 2
# for which the calculations are performed. For example, if max_num_pows_of_2 = 4 , the function does the calculations for 2 ^ 0 , 2 ^ 1 , 2^ 2 , 2 ^ 3
# (the result would have 4 digits in this case)
# Example : binarysplit(6,4)  : [0,1,1,0,0] 

def binarysplit(number_a,max_num_pows_of_2) :
  
  temp = number_a
  split_array = np.zeros((max_num_pows_of_2,))

  for i in reversed(range(0,max_num_pows_of_2)) :
    split_array[i] = int(temp/(2**(i)))
    temp = temp - split_array[i]*(2**(i))

  return split_array

In [ ]:
# This Function applies "binarysplit" function to form a bit-masking matrix, such that in row i, we have split digits for the binary version of decimal number "i"
# (stored as boolean : 0 --> False , 1 --> True)
# Later we will use the bit-masking matrix, as a boolean filter to know which elements would be cherry-picked for each subset.

def bitmask_matrix_constructor(total,n) :

  bitmask_matrix = np.zeros((total,n),dtype=bool)
  for i in range(0,total) :
    bitmask_matrix[i] = binarysplit(i,n)

  return bitmask_matrix

In [ ]:
def subset_array_constructor(myset, no_subset_elements):
  
  # Cardinality of myset
  myset_cardinality = len(myset)

  # Number of all subsets of myset
  total = 2**myset_cardinality

  my_bitmask_matrix = bitmask_matrix_constructor(total,myset_cardinality)

  # Creating an array in which we will store the number of 1 s ( or True values) in each row of bit-masking matrix; 
  # It shows which rows of bit-masking matrix, we should apply as filters to print subsets with a specific number of elements.

  row_sum = np.zeros((total,))
  for i in range(0,total):
    for j in range(0,myset_cardinality):
      row_sum[i] = row_sum[i] + my_bitmask_matrix[i,j]

  # Creating another filter to find the rows in "row_sum" array with value "number_of_elements"(our desired number of elements)

  row_sum_filter = [index for index,value in enumerate(row_sum) if value == no_subset_elements]

  num_of_subsets = int(factorial(myset_cardinality)/(factorial(myset_cardinality-no_subset_elements)*factorial(no_subset_elements)))

  my_subset_array = np.zeros((num_of_subsets,no_subset_elements),dtype = int)
  # applying the "row_sum_filter" to print all the k-element subsets.
  for index,i in enumerate(row_sum_filter) :
    my_subset_array[index,:] = myset[my_bitmask_matrix[i]]
    # print(myset[my_bitmask_matrix[i]])

  return my_subset_array


## Brute-Force Approach

### Brute-Force Approach ( not optimized )

In [ ]:
def BruteForcePDP(L,number_of_cuts) :

  M = np.max(L) 
  my_range = np.arange(1,M) # including 1 , excluding M
  
  length_of_range = len(my_range)
  
  num_of_subsets = int(factorial(length_of_range)/(factorial(length_of_range-(number_of_cuts-2))*factorial(number_of_cuts-2)))
  

  subset_array = subset_array_constructor(my_range,number_of_cuts-2)

  Found_Any_Yet = False 

  for i in range(0,num_of_subsets) :
    X = np.array([0,M])
    X = np.insert(X,1,subset_array[i])

    if np.array_equal(Delta(X),L) :
      Found_Any_Yet = True
      print(X)

  if not Found_Any_Yet :
    print("No Solution!")

In [ ]:
BruteForcePDP(L1,number_of_cuts1)

[ 0  2  4  7 10]
[ 0  3  6  8 10]


In [ ]:
print(Delta(np.array([ 0 , 2 , 4 , 7 , 10])))
print(Delta(np.array([ 0 , 3 , 6 , 8 , 10])))

[ 2.  2.  3.  3.  4.  5.  6.  7.  8. 10.]
[ 2.  2.  3.  3.  4.  5.  6.  7.  8. 10.]


In [ ]:
BruteForcePDP(L2,number_of_cuts2)

No Solution!


In [ ]:
BruteForcePDP(L_takehome,7)

[ 0  3  4  5  6  9 15]
[ 0  6  9 10 11 12 15]


###Brute-Force Approach ( with a minor improvement in terms of time complexity )


In [ ]:
def BruteForcePDP_minor_improvement(L,number_of_cuts) :
  M = np.max(L)

  # note : using "np.unique" method is extremely important, because it removes duplicates from L (L is a multiset so redundancies are allowed)
  # without doing so, we will encounter duplicates when constructing subsets and the final results would be affected too.
  
  my_range = np.unique(np.array(L[0:-1]))    # excluding M from L
  
  length_of_range = len(my_range)
  
  num_of_subsets = int(factorial(length_of_range)/(factorial(length_of_range-(number_of_cuts-2))*factorial(number_of_cuts-2)))

  subset_array = subset_array_constructor(my_range,number_of_cuts-2)

  Found_Any_Yet = False 

  for i in range(0,num_of_subsets) :
    X = np.array([0,M])
    X = np.insert(X,1,subset_array[i])

    if np.array_equal(Delta(X),L) :
      Found_Any_Yet = True
      print(X)

  if not Found_Any_Yet :
    print("No Solution!")

In [ ]:
BruteForcePDP_minor_improvement(L1,number_of_cuts1)

[ 0  2  4  7 10]
[ 0  3  6  8 10]


In [ ]:
BruteForcePDP_minor_improvement(L2,number_of_cuts2)

No Solution!


In [ ]:
BruteForcePDP_minor_improvement(L_takehome,7)

[ 0  3  4  5  6  9 15]
[ 0  6  9 10 11 12 15]


## Efficient Algorithm

In [ ]:
def PDP_efficient(L_input) :
  L = np.sort(L_input)
  width = np.max(L)
  L = np.array([item for item in L if item != width])
  X = np.array([0,width])
  PLACE(L,X,width)
  
  

def PLACE(L,X,width) : 
  
  L_temp = np.array(L)

  if len(L_temp) == 0 : 
    print(X)

    return

  y = np.max(L_temp)
  
  if set(Induced_Difference(y,X)) <= set(L_temp) :
    X1 = np.sort(np.append(X,y))
    L_temp1 = Remove_Elements(L_temp,Induced_Difference(y,X))
    PLACE(L_temp1,X1,width)

  if set(Induced_Difference(width - y,X)) <= set(L_temp) :
    X2 = np.sort(np.append(X,width - y))
    L_temp2 = Remove_Elements(L_temp,Induced_Difference(width - y,X))
    PLACE(L_temp2,X2,width)

  return


    
def Remove_Elements(arr1,arr2) :
  
  C = Counter(arr1) - Counter(arr2)
  temp = []
  
  for i in C.keys() :
    temp.append(C[i]*[i])

  res = np.array([item for sublist in temp for item in sublist])
  return res



In [ ]:
PDP_efficient(L1)

[ 0  3  6  8 10]
[ 0  2  4  7 10]


In [ ]:
PDP_efficient(L2)

In [ ]:
PDP_efficient(L_takehome)

[ 0  6  9 10 11 12 15]
[ 0  6  9 10 11 12 15]
[ 0  3  4  5  6  9 15]
[ 0  3  4  5  6  9 15]
